In [1]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=250
TARGET_DIR='/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250/scfs'
TARGET_SPECIES='Cl'
####################################################



os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()





def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()


av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")





    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f32
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(TARGET_DIR))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s =\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()




endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()



Begin on Thu Feb 15 10:23:21 2018

Working dir:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250/scfs


TEMP:            250
TARGET_DIR:      /Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250/scfs
TARGET_SPECIES:  Cl


There are 1252 efg files in this directory.

199 files were skipped.
between efg.361.out and efg.699.out
display variable "skipped_files" to see all files not opened.
1053 efg files opened without error.

The indices of Cl atoms are:
1 12 18 24 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:245
skipped:429
skipped:458
skipped:459
skipped:460
skipped:462
skipped:463
skipped:464
900 
Report for atomic specie:	1
Working Dir:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250/scfs
<tx>:	 -0.0151140984503 	<ty>:	 0.0732509846021 
 <tx^2>	: 0.00354824599809 	<ty^2>: 0.010981366015 

cq0=	-69.2976
eta0=	0.09687
target:	/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/02

In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

target_atom_labels=	[1, 12, 18, 24]

av_theta_xs=	[-0.015114098450294123, 0.011631754002452121, -0.015953675889721141, 0.014302513777660796]
    
av_theta_ys=	[0.073250984602116889, 0.077572728846410657, 0.07412428586428492, 0.071728448093988248]
  
av_theta_y2s=	[0.010981366014964949, 0.012642177693733386, 0.011296456729599115, 0.012298150251214432]

av_theta_x2s=	[0.0035482459980930945, 0.0023021414529123458, 0.0035044226028891588, 0.0021395900534288358]
 
cq_coef0s =	[0.97820558198041296, 0.97758352128003145, 0.97779868100126754, 0.97834338954303512]

cq_coef1s=	[0.97766554722838717, 0.97683229179704512, 0.97723257024187682, 0.97760534474826544]

eta_coefs=	[]

cq0s=	[-69.2976, -69.2976, -69.2976, -69.2976]

cqprime0s=	[-67.787299137845864, -67.744191824255111, -67.759101876553444, -67.79684887119744]

cqprime1s=	[-67.749876025613887, -67.692133424034921, -67.719871759593488, -67.745704138227396]
 
eta0s=	[0.09687, 0.09687, 0.09687, 0.09687]

etaprimes=	[]

fq0s=	[-33.946616702485429, -33.925029362819288, -33.932496039273765, -33.951399031060546]
 
fqprimes=	[]


fqprimes=	[]
TARGET_DIR='/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250'
os.chdir(TARGET_DIR)
os.chdir(TARGET_DIR)
os.getcwd()
x=md.Md('md.in','md.out')
vol=x.cell_volume
print(TARGET_DIR,"\n","-"*len(TARGET_DIR),"\n")
cq=np.average(cq0s)
eta=np.average(eta0s)
k0=np.average(cq_coef0s)
k1=np.average(cq_coef1s)

print("cell volume:", vol)
print("avg Cq0:", np.average(cq0s),"\t",cq)
print("avg eta0:", np.average(eta0s),"\t",eta)
print("avg coef0:",np.average(cq_coef0s),"\t",k0)
print("avg coef1:",np.average(cq_coef1s),"\t",k1)
print("using coef0:",np.average(cq0s)*np.average(cq_coef0s))
print("using coef1:",np.average(cq0s)*np.average(cq_coef1s))

ffunc=nqr.f32
fraw=ffunc(cq,eta)
f0=ffunc(k0*cq,eta)
f1=ffunc(k1*cq,eta)
expt=34.42
print("raw freq:\t",fraw)
print("expt moross:\t", expt)
print("adjusted f0:\t",f0)
print("adjusted f1:\t",f1)


/Users/altoidnerd/Desktop/paradichlorobenzene3/18.v_dependent_mds/0250 
 ---------------------------------------------------------------------- 

cell volume: 316.729804281
avg Cq0: -69.2976 	 -69.2976
avg eta0: 0.09687 	 0.09687
avg coef0: 0.977982793451 	 0.977982793451
avg coef1: 0.977333938504 	 0.977333938504
using coef0: -67.7718604275
using coef1: -67.7268963369
raw freq:	 -34.7029472412
expt moross:	 34.42
adjusted f0:	 -33.9388852839
adjusted f1:	 -33.9163681049


In [3]:
print("using coef0:",np.average(cq0s)*np.average(cq_coef0s))
print("using coef1:",np.average(cq0s)*np.average(cq_coef1s))

using coef0: -67.7718604275
using coef1: -67.7268963369
